# Imports

In [406]:
!pip install corpus_distance

In [407]:
import corpus_distance

In [408]:
import random
random.seed(6)

# Data loading

In [409]:
CONTENT_DIR = "/home/ilia/Дакументы/PhD HSE/источники/texts/OES"
TOPIC_NORMALISATION = True
SPLIT = 1

Texts (or collections of texts) should be pre-tokenised single strings, (optionally) stored in separate files. Filenames should contain lect name before extension, split by '.'. For example, 'Akimov.Belogornoje.txt', where *Akimov* is a text name, *Belogornoje* is a lect name, and *txt* is an extension.

Texts become dictionary keys, and lects names - its values.

In [410]:
from corpus_distance.data_preprocessing.data_loading import load_data
df = load_data(CONTENT_DIR, SPLIT)

The next stage is transformation of dictionary into a dataframe of the following format:

| index | text | lect |
| -------- | ------- |------- |
| 0 | text1 | lect1 |
| 1 | text2 | lect1 |
| 2 | text1 | lect2 |
| ... | ... | ... |
| m | textN | lectK |

*m* here represents the overall number of texts, *K* - the overall number of lects, and *N* is the number of texts in lect *K*.  

In [411]:
df.head()

,text,lect
0,"се язъ князь ярославъ володимѣричь , сгадавъ с...",Novgorod
1,﻿ⴕ поклонъ и бласловлѣнье • ѡт ѧкова епискупа ...,Polotsk
2,"﻿с азъ, андрѣи данильѥвичь . ажо ми сѧ оучинит...",Polotsk
3,﻿се язъ кнѧзь смоленьскыи федоръ • сѹдилъ есмь...,Smolensk
4,се азъ кнѧзь ѡлександръ и сынъ мои дмитрии с п...,Novgorod


# Data processing

Here we get lect names.

In [412]:
from corpus_distance.cdutils import get_lects_from_dataframe

In [413]:
lects = get_lects_from_dataframe(df)

In [414]:
lects

['Novgorod', 'Polotsk', 'Smolensk']

## Topic modelling

Topic modelling is used to delete topic words that reflect the features of the texts, and not the language.

In [415]:
from corpus_distance.data_preprocessing.topic_modelling import get_topic_words_for_lects, add_thematic_modelling

In [416]:
topic_words = get_topic_words_for_lects(df, lects)

In [417]:
df_without_topics = add_thematic_modelling(df, topic_words, TOPIC_NORMALISATION)

In [418]:
df_without_topics.head()

,text,lect,text_topic_normalised
0,се язъ князь ярославъ володимѣричь сгадавъ пос...,Novgorod,се язъ князь ярославъ володимѣричь сгадавъ пос...
1,﻿ⴕ поклонъ бласловлѣнье ѡт ѧкова епискупа поло...,Polotsk,﻿ⴕ поклонъ бласловлѣнье ѡт ѧкова епискупа поло...
2,"﻿с азъ, андрѣи данильѥвичь ажо ми оучинить быт...",Polotsk,"﻿с азъ, андрѣи данильѥвичь ажо ми оучинить быт..."
3,﻿се язъ кнѧзь смоленьскыи федоръ сѹдилъ есмь б...,Smolensk,﻿се язъ кнѧзь смоленьскыи федоръ сѹдилъ есмь б...
4,се азъ кнѧзь ѡлександръ сынъ мои дмитрии посад...,Novgorod,се азъ кнѧзь ѡлександръ сынъ мои дмитрии посад...


## Vectorisation

I start with creating a model for representing key properties of the lect:

* Its name
* Text it contains, lowercased
* Its alphabet (with obligatory CLS `^` and EOS `$` symbols)
* Amount of enthropy of its alphabet
* Vector for each given symbol of alphabet

In [419]:
from corpus_distance.data_preprocessing.vectorisation import create_vectors_for_lects, gather_vector_information, FastTextParams

In [420]:
vectors_for_lects = create_vectors_for_lects(df_without_topics, FastTextParams(seed=42))

100%|██████████| 3/3 [00:04<00:00,  1.42s/it]


In [421]:
from pprint import pprint

In [422]:
pprint(vectors_for_lects)

{'Novgorod': <corpus_distance.data_preprocessing.vectorisation.Lect object at 0x7e8f330253f0>,
 'Polotsk': <corpus_distance.data_preprocessing.vectorisation.Lect object at 0x7e8f32fa0820>,
 'Smolensk': <corpus_distance.data_preprocessing.vectorisation.Lect object at 0x7e8f32c74760>}


# Date preprocessing

The first stage of data preprocessing is splitting tokens into character 3-grams. The character n-grams help to find coinciding sequences more easily, than tokens or token n-grams. Specifically 3-grams help to underscore the exact places where the change is happening, providing minimal left and right context for each symbol within the sequence. Adding special symbols *^* and *$* to the start and the end of each sequence helps to do this for the first and the last symbol of the given sequence as well.

In [423]:
from corpus_distance.data_preprocessing.shingle_processing import split_lects_by_n_grams

In [424]:
df_with_n_grams = split_lects_by_n_grams(df_without_topics)

New dataframe is in the following format:

| index | lect | n-gram array |
| -------- | ------- |------- |
| 0 | lect1 | n-grams of lect1 |
| 1 | lect1 | n-grams of lect1 |
| ... | ... | ... |
| k | lectK | n-grams of lect lectK |

Here, *k* is overall number of lects.

In [425]:
df_with_n_grams.head()

,lect,n_grams
0,Novgorod,"[^се, се$, ^яз, язъ, зъ$, ^кн, кня, няз, язь, ..."
1,Polotsk,"[^ⴕ$, ^по, пок, окл, кло, лон, онъ, нъ$, ^бл, ..."
2,Smolensk,"[^се, се$, ^яз, язъ, зъ$, ^кн, кнѧ, нѧз, ѧзь, ..."


The next step is to rank n-grams by frequency. The results form *frequency_arranged_n_grams* column of the dataframe.

In [426]:
from corpus_distance.data_preprocessing.frequency_scoring import count_n_grams_frequencies

In [427]:
df_new = count_n_grams_frequencies(df_with_n_grams)

In [428]:
# add information on letter vectors and alphabet information to dataframe

df_new = gather_vector_information(df_new, vectors_for_lects)

In [429]:
df_new.head()

,lect,n_grams,frequency_arranged_n_grams,relative_frequency_n_grams,lect_vectors,lect_info
0,Novgorod,"[^се, се$, ^яз, язъ, зъ$, ^кн, кня, няз, язь, ...","[(оро, 0), (^по, 1), (ть$, 2), (нов, 3), (мъ$,...","[(оро, 0.25), (^по, 0.2504887585532747), (ть$,...","{'^': [0.13765854, -0.04985554, -0.09496442, -...",4.585542
1,Polotsk,"[^ⴕ$, ^по, пок, окл, кло, лон, онъ, нъ$, ^бл, ...","[(^по, 0), (ти$, 1), (пол, 2), (мъ$, 3), (^пр,...","[(^по, 0.25), (ти$, 0.2504916420845624), (пол,...","{'^': [0.00029285578, -0.17601506, -0.08750519...",4.511540
2,Smolensk,"[^се, се$, ^яз, язъ, зъ$, ^кн, кнѧ, нѧз, ѧзь, ...","[(ть$, 0), (ьск, 1), (^см, 2), (смо, 3), (мол,...","[(ть$, 0.25), (ьск, 0.2504646840148699), (^см,...","{'^': [-0.055086426, -0.08944464, 0.04098061, ...",4.500708


# Metrics

First step is to introduce a measure for hybridisation.

One possible measure is scoring Euclidean distance between sum of letter vectors for each n-gram. This results in a loss of order within n-gram, which can yield possible disadvantages (bra === bar), when the measure is used alone; however, when joined with DistRank and Jaro distance, hopefully they yield better results.

Optional normalisation includes using alphabet information difference, calculated via subtraction of the second alphabet information from the first one. This allows to compensate for the cases, when letter from one alphabet may have multiple correspondences in the other, depending on the context. Direct (and not reversed, `1 - X`) measure is better, because the more information one alphabet carries, when contrasted to the other, the more possible one-to-many correspondences there are, the more distortions in vectors there are, the more normalisation is needed.

Final normalisation includes traditional split by maximal length of two strings, introduced in Holman et al. (2008).

In [430]:
from corpus_distance.distance_measurement.string_similarity import *
from corpus_distance.distance_measurement.hybridisation import HybridisationParameters

In [431]:
# assigning global values
# group of languages  and its outgroup
GROUP = "Old East Slavic"
OUTGROUP = "Novgorod"

# if hybrid metrics aids DistRank
HYBRIDISATION = True
# if hybrid values join DistRank values in a single array, or they both are
# independent values, equally contributing to the final metric
HYBRIDISATION_AS_ARRAY = True

# if distrank normalisation includes soerensen coefficient
SOERENSEN_NORMALISATION = True

# choose a metric for hybridisation
HYBRID = weighted_jaro_winkler_wrapper

# if string similarity measure includes correction by
# difference in the information that alphabets carry
ALPHABET_NORMALISATION = True

# metric description
METRICS = f"{GROUP}-{SPLIT}-{TOPIC_NORMALISATION}-DistRank-{SOERENSEN_NORMALISATION}-{HYBRIDISATION}-{HYBRIDISATION_AS_ARRAY}-{HYBRID.__name__}-{ALPHABET_NORMALISATION}"

In [432]:
hybridisation_parameters = HybridisationParameters(HYBRIDISATION, SOERENSEN_NORMALISATION, HYBRIDISATION_AS_ARRAY, HYBRID, ALPHABET_NORMALISATION)

In [433]:
METRICS

'Old East Slavic-1-True-DistRank-True-True-True-weighted_jaro_winkler_wrapper-True'

In [434]:
from corpus_distance.distance_measurement.metrics_pipeline import score_metrics_for_corpus_dataset

In [435]:
# declare arrays
# calculate distances for each pair of lects
overall_results = score_metrics_for_corpus_dataset(df_new, "/home/ilia/Дакументы/vector_analysis/experiment_results/exp_233", METRICS, hybridisation_parameters)

100%|██████████| 631/631 [00:11<00:00, 57.14it/s]


# Clusterisation

The final step is to cluster the lects into groups, and to decide, whether the method works correctly.

In [436]:
from corpus_distance.clusterisation.clusterisation import ClusterisationParameters, clusterise_lects_from_distance_matrix
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor

In [437]:
cluster_params = ClusterisationParameters(lects, OUTGROUP, GROUP, METRICS, DistanceTreeConstructor().upgma, "/home/ilia/Дакументы/vector_analysis/experiment_results/exp_233")

In [438]:
clusterise_lects_from_distance_matrix(overall_results, cluster_params)